# Who will be the next NBA MVP?
### Modeling MVP vote share using player and team stats 

Of all the awards in the NBA, the MVP award is the most coveted. Being the "Most Valuable Player" is a sign of a player's standing in the league; this distinction often goes to the best player, and almost always goes to a top 5 player. While "most valuable" can mean different things to voters, coaches, and players, there are some distinct commonalities between all players in the history of the award. 

The goal of this analysis is to use machine learning to predict the MVP vote share of a player given their season stats. The model will be tested on a random subset of all seasons, as well as the current 2020 season.

### Table of Contents
* [Data Collection](#datacollection)
    * [Importing](#import)
    * [Cleaning](#clean)
* [Creating the Pipeline](#pipeline)
    * [Team Stats](#teamstats)
    * [Adding MVP Share](#mvpshare)
* [Training and Testing](#ttsplit)
    * [Building the Design Matrix](#designmatrix)
    * [Training-Test Split](#split)
    * [2019-2020 Season Evaluation Set](#2020stats)
* [Cross-Validation](#cv)
* [Conclusion](#conclusion)

<a id='datacollection'></a>
# Data Collection

The first step requires collection of two main categories of data: the MVP shares by season for each player receiving a vote, and the player's season stats. The source of all this data will be from www.basketball-reference.com, a free-to-use platform that compiles all NBA data that exists, including award voting results, play-by-play data, and most importantly, team and player stats per season. 

I was able to find a spreadsheet compiled by an user of the website that contains MVP voting share referenced by season and player for all seasons. This greatly reduced the time needed to collect data, as there is no automated way to do this from the BballRef website. The source for this spreadsheet is linked in [this post](https://www.reddit.com/r/nba/comments/cqvjsi/octhe_10_players_with_the_highest_mvp_voting/).

The features of the model will be player stats, which will be obtained through a web scraping package made for BballRef. The package's documentation is linked here: [basketball_reference_scraper](https://github.com/vishaalagartha/basketball_reference_scraper/blob/master/API.md).

Below, the necessary python packages are imported.

In [54]:
#%pip install --upgrade pip

# data wrangling packages
import pandas as pd
import numpy as np

# web scraper
#%pip install basketball-reference-scraper
from basketball_reference_scraper.players import get_stats, get_game_logs, get_player_headshot
from basketball_reference_scraper.teams import get_roster, get_team_stats, get_opp_stats, get_roster_stats, get_team_misc

# data visualization packages
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# data modeling packages
from sklearn import linear_model as lm
from sklearn.model_selection import train_test_split, KFold

<a id='import'></a>
## Importing

The MVP share dataset is imported below as a pandas DataFrame. Additionally, the web scraper is tested.

In [55]:
mvp_share = pd.read_csv('mvp_share_by_season.csv')
mvp_share.head()

,Player,Total,1956,1957,1958,1959,1960,1961,1962,1963,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Bob Pettit,3.419,0.413,0.263,0.243,0.773,0.268,0.329,0.073,0.166,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Paul Arizin,0.494,0.263,0.113,0.020,0.095,NaN,0.003,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bob Cousy,0.873,0.138,0.288,0.045,0.173,0.167,0.045,0.007,0.010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mel Hutchins,0.126,0.113,0.013,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dolph Schayes,0.730,0.025,0.100,0.495,0.063,0.015,0.032,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
kd_stats = get_stats('Kevin Durant', stat_type='PER_GAME', playoffs=False, career=False)
kd_stats

,SEASON,AGE,TEAM,LEAGUE,POS,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2007-08,19.0,SEA,NBA,SG,80.0,80.0,34.6,7.3,17.1,...,0.873,0.9,3.5,4.4,2.4,1.0,0.9,2.9,1.5,20.3
1,2008-09,20.0,OKC,NBA,SF,74.0,74.0,39.0,8.9,18.8,...,0.863,1.0,5.5,6.5,2.8,1.3,0.7,3.0,1.8,25.3
2,2009-10,21.0,OKC,NBA,SF,82.0,82.0,39.5,9.7,20.3,...,0.900,1.3,6.3,7.6,2.8,1.4,1.0,3.3,2.1,30.1
3,2010-11,22.0,OKC,NBA,SF,78.0,78.0,38.9,9.1,19.7,...,0.880,0.7,6.1,6.8,2.7,1.1,1.0,2.8,2.0,27.7
4,2011-12,23.0,OKC,NBA,SF,66.0,66.0,38.6,9.7,19.7,...,0.860,0.6,7.4,8.0,3.5,1.3,1.2,3.8,2.0,28.0
5,2012-13,24.0,OKC,NBA,SF,81.0,81.0,38.5,9.0,17.7,...,0.905,0.6,7.3,7.9,4.6,1.4,1.3,3.5,1.8,28.1
6,2013-14,25.0,OKC,NBA,SF,81.0,81.0,38.5,10.5,20.8,...,0.873,0.7,6.7,7.4,5.5,1.3,0.7,3.5,2.1,32.0
7,2014-15,26.0,OKC,NBA,SF,27.0,27.0,33.8,8.8,17.3,...,0.854,0.6,6.0,6.6,4.1,0.9,0.9,2.7,1.5,25.4
8,2015-16,27.0,OKC,NBA,SF,72.0,72.0,35.8,9.7,19.2,...,0.898,0.6,7.6,8.2,5.0,1.0,1.2,3.5,1.9,28.2
9,2016-17,28.0,GSW,NBA,PF,62.0,62.0,33.4,8.9,16.5,...,0.875,0.6,7.6,8.3,4.8,1.1,1.6,2.2,1.9,25.1


<a id='clean'></a>
## Cleaning

I'll start with some preliminary EDA to determine that the values in the MVP share DataFrame are all in fact numbers.

In [57]:
mvp_share = mvp_share.set_index('Player')
mvp_share.head()

,Total,1956,1957,1958,1959,1960,1961,1962,1963,1964,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Player,,,,,,,,,,,,,,,,,,,,,
Bob Pettit,3.419,0.413,0.263,0.243,0.773,0.268,0.329,0.073,0.166,0.144,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Paul Arizin,0.494,0.263,0.113,0.020,0.095,NaN,0.003,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bob Cousy,0.873,0.138,0.288,0.045,0.173,0.167,0.045,0.007,0.010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mel Hutchins,0.126,0.113,0.013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dolph Schayes,0.730,0.025,0.100,0.495,0.063,0.015,0.032,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The index is changed to player names to make indexing easier.

In [58]:
print(mvp_share.loc['Kevin Durant', 'Total'], type(mvp_share.loc['Kevin Durant', 'Total']))
print(mvp_share.loc['Kevin Durant', '2014'], type(mvp_share.loc['Kevin Durant', '2014']))

3.2089999999999996 <class 'numpy.float64'>
0.986 <class 'numpy.float64'>


It looks like there are mostly NaN values in the DataFrame. In order to make comparisons easier later, we'll impute the value -1 to distinguish these values.

In [59]:
mvp_share = mvp_share.fillna(-1)
mvp_share.head()

,Total,1956,1957,1958,1959,1960,1961,1962,1963,1964,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Player,,,,,,,,,,,,,,,,,,,,,
Bob Pettit,3.419,0.413,0.263,0.243,0.773,0.268,0.329,0.073,0.166,0.144,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Paul Arizin,0.494,0.263,0.113,0.020,0.095,-1.000,0.003,-1.000,-1.000,-1.000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Bob Cousy,0.873,0.138,0.288,0.045,0.173,0.167,0.045,0.007,0.010,-1.000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Mel Hutchins,0.126,0.113,0.013,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Dolph Schayes,0.730,0.025,0.100,0.495,0.063,0.015,0.032,-1.000,-1.000,-1.000,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


Additionally, I'll drop all columns from the years 1956 through 1980. This is done for a number of reasons: the MVP award was voted on by players until the 1980-1981 season, at which point it became a panel of sportswriters. Also, the 3pt line was added in the 1979-1980 season. This means I'll also have to drop all players who did not receive any MVP votes in seasons 1980-1981 and beyond. After looking at the original data, this is all players up to and including Billy Cuningham. This is confirmed in the cell directly below.

In [60]:
test = mvp_share.loc[:'Billy Cuningham', '1981':]
a = [test[i].value_counts() for i in test.columns]
a

[-1.0    47
 Name: 1981, dtype: int64,
 -1.0    47
 Name: 1982, dtype: int64,
 -1.0    47
 Name: 1983, dtype: int64,
 -1.0    47
 Name: 1984, dtype: int64,
 -1.0    47
 Name: 1985, dtype: int64,
 -1.0    47
 Name: 1986, dtype: int64,
 -1.0    47
 Name: 1987, dtype: int64,
 -1.0    47
 Name: 1988, dtype: int64,
 -1.0    47
 Name: 1989, dtype: int64,
 -1.0    47
 Name: 1990, dtype: int64,
 -1.0    47
 Name: 1991, dtype: int64,
 -1.0    47
 Name: 1992, dtype: int64,
 -1.0    47
 Name: 1993, dtype: int64,
 -1.0    47
 Name: 1994, dtype: int64,
 -1.0    47
 Name: 1995, dtype: int64,
 -1.0    47
 Name: 1996, dtype: int64,
 -1.0    47
 Name: 1997, dtype: int64,
 -1.0    47
 Name: 1998, dtype: int64,
 -1.0    47
 Name: 1999, dtype: int64,
 -1.0    47
 Name: 2000, dtype: int64,
 -1.0    47
 Name: 2001, dtype: int64,
 -1.0    47
 Name: 2002, dtype: int64,
 -1.0    47
 Name: 2003, dtype: int64,
 -1.0    47
 Name: 2004, dtype: int64,
 -1.0    47
 Name: 2005, dtype: int64,
 -1.0    47
 Name: 2006, 

In [61]:
mvp_share = mvp_share.drop(columns=[str(i) for i in np.arange(1956, 1981)])
mvp_share = mvp_share.loc['Kareem Abdul-Jabbar':, :]
mvp_share

,Total,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Player,,,,,,,,,,,,,,,,,,,,,
Kareem Abdul-Jabbar,6.104,0.414,0.045,0.02,0.201,0.264,0.173,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.000
Walt Frazier,0.170,-1.000,-1.000,-1.00,-1.000,-1.000,-1.000,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.000
Conny Hawkins,0.041,-1.000,-1.000,-1.00,-1.000,-1.000,-1.000,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.000
Lou Hudson,0.032,-1.000,-1.000,-1.00,-1.000,-1.000,-1.000,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.000
Bill Bridges,0.015,-1.000,-1.000,-1.00,-1.000,-1.000,-1.000,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jimmy Butler,0.005,-1.000,-1.000,-1.00,-1.000,-1.000,-1.000,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.005,-1.000
Joel Embiid,0.053,-1.000,-1.000,-1.00,-1.000,-1.000,-1.000,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.004,0.049
Victor Oladipo,0.002,-1.000,-1.000,-1.00,-1.000,-1.000,-1.000,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.002,-1.000


<a id='pipeline'></a>
# Creating the Pipeline

Now that I have the cleaned data and know how the scraper works, it's time to start building the data processing pipeline. This will allow me generalize a set of functions that can work on any season/player and build the design matrix for linear regression.

<a id='teamstats'></a>
## Team Stats

The get_stats function of the web scraper is very useful for individual player stats, which will comprise the vast majority of the necessary features. However, team record plays a huge role, as only two MVP's since the 1980-1981 season have had team records with less than 50 wins. To account for this, I'll create a function that is essentially a copy of get_stats, but adds a column with the team's win percentage. 

I'll also drop some irrelevant columns like Team, League, GS, etc., as they will have little to no bearing on the prediction and may lead to overfitting.

In [62]:
kd_stats.columns

Index(['SEASON', 'AGE', 'TEAM', 'LEAGUE', 'POS', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [63]:
kd_stats.head()

,SEASON,AGE,TEAM,LEAGUE,POS,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2007-08,19.0,SEA,NBA,SG,80.0,80.0,34.6,7.3,17.1,...,0.873,0.9,3.5,4.4,2.4,1.0,0.9,2.9,1.5,20.3
1,2008-09,20.0,OKC,NBA,SF,74.0,74.0,39.0,8.9,18.8,...,0.863,1.0,5.5,6.5,2.8,1.3,0.7,3.0,1.8,25.3
2,2009-10,21.0,OKC,NBA,SF,82.0,82.0,39.5,9.7,20.3,...,0.900,1.3,6.3,7.6,2.8,1.4,1.0,3.3,2.1,30.1
3,2010-11,22.0,OKC,NBA,SF,78.0,78.0,38.9,9.1,19.7,...,0.880,0.7,6.1,6.8,2.7,1.1,1.0,2.8,2.0,27.7
4,2011-12,23.0,OKC,NBA,SF,66.0,66.0,38.6,9.7,19.7,...,0.860,0.6,7.4,8.0,3.5,1.3,1.2,3.8,2.0,28.0


In [72]:
def get_stats_cleaned(name):
    ''' This function will be used to return player career stats with a cleaned SEASON column,
    added W/L percentage column, and removed TEAM, LEAGUE, GS, TRB, and PF columns.
    '''
    
    # get initial dataframe
    original = get_stats(name, stat_type='PER_GAME', playoffs=False, career=False)
    
    # clean seasons
    cleaned_season = []
    for i in original['SEASON']:
        if i == '1999-00':
            cleaned_season.append('2000')
        elif int(i.split('-')[0]) < 1999:
            cleaned_season.append('19' + i.split('-')[1])
        else:
            cleaned_season.append('20' + i.split('-')[1])
    original['SEASON'] = cleaned_season
    
    # add team W/L
    win_percentage = []
    for i in np.arange(original.shape[0]):
        wins = get_team_misc(original.iloc[i, 2], int(original.iloc[i, 0])).loc['W']
        losses = get_team_misc(original.iloc[i, 2], int(original.iloc[i, 0])).loc['L']
        win_percentage.append(wins / (wins + losses))
    original['Win %'] = win_percentage
    
    # drop necessary columns
    original = original.drop(columns=['TEAM', 'LEAGUE', 'ORB', 'DRB', 'PF'])
    
    # set index to season for next function to work
    original.set_index('SEASON', inplace=True)
    
    # drop 2020 data
    if '2020' in original.index.to_list():
        original = original.drop('2020')
        
    return original

I'll test the function below.

In [65]:
get_stats_cleaned('Kevin Durant')

,AGE,POS,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,FTA,FT%,ORB,DRB,AST,STL,BLK,TOV,PTS,Win %
SEASON,,,,,,,,,,,,,,,,,,,,,
2008,19.0,SG,80.0,34.6,7.3,17.1,0.430,0.7,2.6,0.288,...,5.6,0.873,0.9,3.5,2.4,1.0,0.9,2.9,20.3,0.243902
2009,20.0,SF,74.0,39.0,8.9,18.8,0.476,1.3,3.1,0.422,...,7.1,0.863,1.0,5.5,2.8,1.3,0.7,3.0,25.3,0.280488
2010,21.0,SF,82.0,39.5,9.7,20.3,0.476,1.6,4.3,0.365,...,10.2,0.900,1.3,6.3,2.8,1.4,1.0,3.3,30.1,0.609756
2011,22.0,SF,78.0,38.9,9.1,19.7,0.462,1.9,5.3,0.350,...,8.7,0.880,0.7,6.1,2.7,1.1,1.0,2.8,27.7,0.670732
2012,23.0,SF,66.0,38.6,9.7,19.7,0.496,2.0,5.2,0.387,...,7.6,0.860,0.6,7.4,3.5,1.3,1.2,3.8,28.0,0.712121
2013,24.0,SF,81.0,38.5,9.0,17.7,0.510,1.7,4.1,0.416,...,9.3,0.905,0.6,7.3,4.6,1.4,1.3,3.5,28.1,0.731707
2014,25.0,SF,81.0,38.5,10.5,20.8,0.503,2.4,6.1,0.391,...,9.9,0.873,0.7,6.7,5.5,1.3,0.7,3.5,32.0,0.719512
2015,26.0,SF,27.0,33.8,8.8,17.3,0.510,2.4,5.9,0.403,...,6.3,0.854,0.6,6.0,4.1,0.9,0.9,2.7,25.4,0.548780
2016,27.0,SF,72.0,35.8,9.7,19.2,0.505,2.6,6.7,0.387,...,6.9,0.898,0.6,7.6,5.0,1.0,1.2,3.5,28.2,0.670732


<a id='mvpshare'></a>
## Adding MVP Share

Now that we have our cleaned data with all our necessary features for the design matrix, its time to add the corresponding MVP share for each season. This will allow for creation of the true voting share values.

In [76]:
def add_mvp_share(player):
    ''' This function retrieves a players seasonal stats and adds the MVP vote share they received for each season.
    '''
    
    # get cleaned dataframe
    original = get_stats_cleaned(player)
    
    # add mvp voting share
    mvp_shares = []
    # drop seasons with 0 mvp votes
    no_votes = []
    
    for i in original.index:
        if int(i) < 1981:
            vote = -1
        else:
            vote = mvp_share.loc[player, i]
       
        mvp_shares.append(vote)
        if vote < 0:
            no_votes.append(i)
            
    original['MVP Share'] = mvp_shares    
    original = original.drop(index=no_votes).reset_index()
    
    return original

I'll test the function below with a few notable players.

In [67]:
add_mvp_share('Kevin Durant')

,SEASON,AGE,POS,G,MP,FG,FGA,FG%,3P,3PA,...,FT%,ORB,DRB,AST,STL,BLK,TOV,PTS,Win %,MVP Share
0,2010,21.0,SF,82.0,39.5,9.7,20.3,0.476,1.6,4.3,...,0.900,1.3,6.3,2.8,1.4,1.0,3.3,30.1,0.609756,0.495
1,2011,22.0,SF,78.0,38.9,9.1,19.7,0.462,1.9,5.3,...,0.880,0.7,6.1,2.7,1.1,1.0,2.8,27.7,0.670732,0.157
2,2012,23.0,SF,66.0,38.6,9.7,19.7,0.496,2.0,5.2,...,0.860,0.6,7.4,3.5,1.3,1.2,3.8,28.0,0.712121,0.735
3,2013,24.0,SF,81.0,38.5,9.0,17.7,0.510,1.7,4.1,...,0.905,0.6,7.3,4.6,1.4,1.3,3.5,28.1,0.731707,0.632
4,2014,25.0,SF,81.0,38.5,10.5,20.8,0.503,2.4,6.1,...,0.873,0.7,6.7,5.5,1.3,0.7,3.5,32.0,0.719512,0.986
5,2016,27.0,SF,72.0,35.8,9.7,19.2,0.505,2.6,6.7,...,0.898,0.6,7.6,5.0,1.0,1.2,3.5,28.2,0.670732,0.112
6,2017,28.0,PF,62.0,33.4,8.9,16.5,0.537,1.9,5.0,...,0.875,0.6,7.6,4.8,1.1,1.6,2.2,25.1,0.817073,0.002
7,2018,29.0,SF,68.0,34.2,9.3,18.0,0.516,2.5,6.1,...,0.889,0.5,6.4,5.4,0.7,1.8,3.0,26.4,0.707317,0.065
8,2019,30.0,SF,78.0,34.6,9.2,17.7,0.521,1.8,5.0,...,0.885,0.4,5.9,5.9,0.7,1.1,2.9,26.0,0.695122,0.025


In [68]:
add_mvp_share('LeBron James')

,SEASON,AGE,POS,G,MP,FG,FGA,FG%,3P,3PA,...,FT%,ORB,DRB,AST,STL,BLK,TOV,PTS,Win %,MVP Share
0,2004,19.0,SG,79.0,39.5,7.9,18.9,0.417,0.8,2.7,...,0.754,1.3,4.2,5.9,1.6,0.7,3.5,20.9,0.426829,0.009
1,2005,20.0,SF,80.0,42.4,9.9,21.1,0.472,1.4,3.9,...,0.750,1.4,6.0,7.2,2.2,0.7,3.3,27.2,0.512195,0.073
2,2006,21.0,SF,79.0,42.5,11.1,23.1,0.480,1.6,4.8,...,0.738,0.9,6.1,6.6,1.6,0.8,3.3,31.4,0.609756,0.550
3,2007,22.0,SF,78.0,40.9,9.9,20.8,0.476,1.3,4.0,...,0.698,1.1,5.7,6.0,1.6,0.7,3.2,27.3,0.609756,0.142
4,2008,23.0,SF,75.0,40.4,10.6,21.9,0.484,1.5,4.8,...,0.712,1.8,6.1,7.2,1.8,1.1,3.4,30.0,0.548780,0.348
5,2009,24.0,SF,81.0,37.7,9.7,19.9,0.489,1.6,4.7,...,0.780,1.3,6.3,7.2,1.7,1.1,3.0,28.4,0.804878,0.969
6,2010,25.0,SF,76.0,39.0,10.1,20.1,0.503,1.7,5.1,...,0.767,0.9,6.4,8.6,1.6,1.0,3.4,29.7,0.743902,0.980
7,2011,26.0,SF,79.0,38.8,9.6,18.8,0.510,1.2,3.5,...,0.759,1.0,6.5,7.0,1.6,0.6,3.6,26.7,0.707317,0.431
8,2012,27.0,SF,62.0,37.5,10.0,18.9,0.531,0.9,2.4,...,0.771,1.5,6.4,6.2,1.9,0.8,3.4,27.1,0.696970,0.888
9,2013,28.0,PF,76.0,37.9,10.1,17.8,0.565,1.4,3.3,...,0.753,1.3,6.8,7.3,1.7,0.9,3.0,26.8,0.804878,0.998


It looks as though the function is working properly: we now have a dataframe with all relevant player stats and their mvp share in each season they received votes.

<a id='ttsplit'></a>
# Training and Testing

Now that we have a pipeline of functions to source our training data, it's time to begin building the design matrix, separate the data into training and test, as well as create an evaluation set consisting of the current season's stats.

<a id='designmatrix'></a>
## Building the Design Matrix

The design matrix will essentially be the resulting dataframe after calling add_mvp_votes but with the SEASON columns dropped. Additionally, the categorical variable of POS (player position) will need to be one-hot encoded.

In [74]:
def design_matrix(players):
    # get data
    temp = add_mvp_share(players.reset_index().iloc[0, 0])
    for player in players.iloc[1:, :].index.to_list:
        print(player)
        temp = temp.append(add_mvp_share(player))
    
    # one-hot encode position
    encoded = pd.get_dummies(temp['POS'], drop_first=False)
    
    return encoded

In [83]:
get_stats('Kareem Abdul-Jabbar').columns

Index(['SEASON', 'AGE', 'TEAM', 'LEAGUE', 'POS', 'G', 'MP', 'FG', 'FGA', 'FG%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'TRB', 'AST', 'PF',
       'PTS'],
      dtype='object')

In [82]:
get_stats('Larry Bird').columns

Index(['SEASON', 'AGE', 'TEAM', 'LEAGUE', 'POS', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [77]:
add_mvp_share('Kareem Abdul-Jabbar')

,SEASON,AGE,POS,G,MP,FG,FGA,FG%,2P,2PA,2P%,eFG%,FT,FTA,FT%,AST,PTS,Win %,MVP Share
0,1981,33.0,C,80.0,37.2,10.5,18.2,0.574,10.5,18.2,0.574,0.574,5.3,6.9,0.766,3.4,26.2,0.658537,0.414
1,1982,34.0,C,76.0,35.2,9.9,17.1,0.579,9.9,17.1,0.580,0.579,4.1,5.8,0.706,3.0,23.9,0.695122,0.045
2,1983,35.0,C,79.0,32.3,9.1,15.5,0.588,9.1,15.5,0.589,0.588,3.5,4.7,0.749,2.5,21.8,0.707317,0.020
3,1984,36.0,C,80.0,32.8,9.0,15.5,0.578,9.0,15.5,0.579,0.578,3.6,4.9,0.723,2.6,21.5,0.658537,0.201
4,1985,37.0,C,79.0,33.3,9.2,15.3,0.599,9.2,15.3,0.600,0.599,3.7,5.0,0.732,3.2,22.0,0.756098,0.264
5,1986,38.0,C,79.0,33.3,9.6,16.9,0.564,9.6,16.9,0.565,0.564,4.3,5.6,0.765,3.5,23.4,0.756098,0.173


In [43]:
design_matrix(mvp_share)

AttributeError: 'NoneType' object has no attribute 'replace'

<a id='split'></a>
## Training-Test Split

<a id='2020stats'></a>
## 2019-2020 Season Evaluation Set

<a id='cv'></a>
# Cross-Validation

<a id='conclusion'></a>
# Conclusion